### - Sobhan Moradian Daghigh
#### - 5/24/2022
#### - Convex - HW02 - Q10

In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def makeA1(a, b, c, n):
    arr = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i - j == 1:
                arr[i, j] = a
            elif i == j:
                arr[i, j] = b
            elif i - j == -1:
                arr[i, j] = c
    return arr                

In [3]:
def makeA2(n):
    arr = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            arr[i, j] = 1 / (i + j + 1)
    return arr      

In [4]:
makeA1(a=-1, b=4, c=-1, n=5)

array([[ 4., -1.,  0.,  0.,  0.],
       [-1.,  4., -1.,  0.,  0.],
       [ 0., -1.,  4., -1.,  0.],
       [ 0.,  0., -1.,  4., -1.],
       [ 0.,  0.,  0., -1.,  4.]])

In [5]:
makeA2(n=5)

array([[1.        , 0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.5       , 0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714],
       [0.25      , 0.2       , 0.16666667, 0.14285714, 0.125     ],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111]])

In [6]:
def opt_objective(x, A, b):
    return np.subtract(np.multiply(0.5, np.matmul(np.matmul(x.T, A), x)), np.matmul(b.T, x))

In [7]:
def get_norm(x):
    return np.linalg.norm(x)

In [8]:
def get_gradient(x, A, b):
    return np.subtract(np.matmul(A, x), b)

In [9]:
def check_first_wolfe(x, A, b, c, alpha, pk):
    if opt_objective(x + alpha * pk, A, b) > opt_objective(x, A, b) + (c * alpha * -np.matmul(get_gradient(x, A, b).T, get_gradient(x, A, b))):
        return True

In [10]:
def get_B(x, x_prev, bk, A, b):
    # According to the Wikipedia
    sk = (x - x_prev).reshape(x.shape[0], 1)
    yk = (get_gradient(x, A, b) - get_gradient(x_prev, A, b)).reshape(x.shape[0], 1)
    bk1 = np.add(bk, 
                 np.subtract(yk @ yk.T / (yk.T @ sk)[0][0], 
                                 (bk @ sk @ sk.T @ bk) / (sk.T @ bk @ sk)[0][0]))
    return bk

In [11]:
def backtracking(x, A, b, alpha, pk, r, c):
    while check_first_wolfe(x=x, A=A, b=b, c=c, alpha=alpha, pk=pk):
        alpha *= r
    return alpha

In [12]:
def steepest(x, A, b, alpha, r, c, iterations):
    for i in range(iterations):
        gradient = get_gradient(x, A, b)
        pk = -gradient
        alpha = backtracking(x, A, b, alpha, pk, r, c)
        x -= np.multiply(alpha, gradient)
    return x

In [13]:
def newton(x, A, b, alpha, r, c, iterations):
    for i in range(iterations):
        gradient = get_gradient(x, A, b)
        hessian = A
        pk = - np.matmul(np.linalg.inv(hessian), gradient)
        alpha = backtracking(x, A, b, alpha, pk, r, c)
        x += alpha * pk
    return x

In [14]:
def BFGS(x, A, b, alpha, r, c, iterations, n):
    bk = np.eye(n)
    for i in range(iterations):
        gradient = get_gradient(x, A, b)
        pk = - np.matmul(np.linalg.inv(bk), gradient)
        alpha = backtracking(x, A, b, alpha, pk, r, c)
        x_prev = x.copy()
        x += alpha * pk
        bk = get_B(x, x_prev, bk, A, b)
    return x

### Part A.

In [15]:
ns = [40, 400]

#### Steepest:

In [16]:
steepest_errors = []
for n in ns:
    As = []
    As.append(makeA1(a=-1, b=4, c=-1, n=n))
    As.append(makeA2(n=n))
    for A in As:
        x = np.zeros(n)
        b = np.ones(n)
        alpha = 0.5
        r = 0.1
        c = 0.3
        iterations = 200
        x_star = steepest(x, A, b, alpha, r, c, iterations)
        error = get_norm(get_gradient(x_star, A, b))  
        steepest_errors.append(error)

In [17]:
steepest_errors

[5.881222812854774e-07,
 0.24894885463792382,
 1.0310992602026099e-06,
 0.8093246608581389]

#### Newtons

In [18]:
newton_errors = []
for n in ns:
    As = []
    As.append(makeA1(a=-1, b=4, c=-1, n=n))
    As.append(makeA2(n=n))
    for i, A in enumerate(As):
        x = np.zeros(n)
        b = np.ones(n)
        alpha = [0.5, 0.17]
        r = [0.1, 0.1]
        c = [0.3, 20]
        iterations = 100
        x_star = newton(x, A, b, alpha[i], r[i], c[i], iterations)
        error = get_norm(get_gradient(x_star, A, b))  
        newton_errors.append(error)

In [19]:
newton_errors

[1.884864367208147e-07,
 6.324555320336759,
 2.2649764954585424e-06,
 15.162933756605419]

#### BFGS

In [20]:
bfgs_errors = []
for n in ns:
    As = []
    As.append(makeA1(a=-1, b=4, c=-1, n=n))
    As.append(makeA2(n=n))
    for A in As:
        x = np.zeros(n).astype(np.float32)
        b = np.ones(n)
        alpha = 0.5
        r = 0.1
        c = 0.3
        iterations = 200
        x_star = BFGS(x, A, b, alpha, r, c, iterations, n)
        error = get_norm(get_gradient(x_star, A, b))  
        bfgs_errors.append(error)

C:\Users\VIRGIL~1\AppData\Local\Temp/ipykernel_14436/3543894875.py:6: RuntimeWarning: invalid value encountered in true_divide
  np.subtract(yk @ yk.T / (yk.T @ sk)[0][0],
C:\Users\VIRGIL~1\AppData\Local\Temp/ipykernel_14436/3543894875.py:7: RuntimeWarning: invalid value encountered in true_divide
  (bk @ sk @ sk.T @ bk) / (sk.T @ bk @ sk)[0][0]))


In [21]:
bfgs_errors

[1.1603785952288976e-06,
 0.2489488624939514,
 1.1603785952288976e-06,
 0.8093246543796154]

### Part B.

In [22]:
eps = 1e-6

#### Steepest + eps

In [23]:
new_steepest_errors = []
for n in ns:
    As = []
    As.append(makeA1(a=-1, b=4, c=-1, n=n) + eps)
    As.append(makeA2(n=n) + eps)
    for A in As:
        x = np.zeros(n)
        b = np.ones(n)
        alpha = 0.5
        r = 0.1
        c = 0.3
        iterations = 200
        x_star = steepest(x, A, b, alpha, r, c, iterations)
        error = get_norm(get_gradient(x_star, A, b))  
        new_steepest_errors.append(error)

In [24]:
new_steepest_errors

[4.704449448941997e-07,
 0.2489247091698735,
 8.936064101334794e-06,
 0.8085477676092931]

In [25]:
for error, new_error in zip(steepest_errors, new_steepest_errors):
    print(get_norm(np.subtract(error, new_error)))

1.1767733639127766e-07
2.4145468050323915e-05
7.904964841132185e-06
0.000776893248845778


#### Newton + eps

In [26]:
new_newton_errors = []
for n in ns:
    As = []
    As.append(makeA1(a=-1, b=4, c=-1, n=n) + eps)
    As.append(makeA2(n=n) + eps)
    for i, A in enumerate(As):
        x = np.zeros(n)
        b = np.ones(n)
        alpha = [0.5, 0.17]
        r = [0.1, 0.1]
        c = [0.3, 20]
        iterations = 100
        x_star = newton(x, A, b, alpha[i], r[i], c[i], iterations)
        error = get_norm(get_gradient(x_star, A, b))  
        new_newton_errors.append(error)

In [30]:
new_newton_errors

[3.769540264625242e-07,
 6.324555320336759,
 9.53669077324859e-06,
 94.39642645059253]

In [27]:
for error, new_error in zip(newton_errors, new_newton_errors):
    print(get_norm(np.subtract(error, new_error)))

1.884675897417095e-07
0.0
7.271714277790048e-06
79.2334926939871


#### BFGS

In [28]:
new_bfgs_errors = []
for n in ns:
    As = []
    As.append(makeA1(a=-1, b=4, c=-1, n=n) + eps)
    As.append(makeA2(n=n) + eps)
    for A in As:
        x = np.zeros(n).astype(np.float32)
        b = np.ones(n)
        alpha = 0.5
        r = 0.1
        c = 0.3
        iterations = 200
        x_star = BFGS(x, A, b, alpha, r, c, iterations, n)
        error = get_norm(get_gradient(x_star, A, b))  
        new_bfgs_errors.append(error)

C:\Users\VIRGIL~1\AppData\Local\Temp/ipykernel_14436/3543894875.py:6: RuntimeWarning: invalid value encountered in true_divide
  np.subtract(yk @ yk.T / (yk.T @ sk)[0][0],
C:\Users\VIRGIL~1\AppData\Local\Temp/ipykernel_14436/3543894875.py:7: RuntimeWarning: invalid value encountered in true_divide
  (bk @ sk @ sk.T @ bk) / (sk.T @ bk @ sk)[0][0]))


In [31]:
new_bfgs_errors

[1.5938068890557663e-06,
 0.24892469994388447,
 6.937263777844877e-06,
 0.8085477621732413]

In [29]:
for error, new_error in zip(bfgs_errors, new_bfgs_errors):
    print(get_norm(np.subtract(error, new_error)))

4.334282938268687e-07
2.4162550066930688e-05
5.77688518261598e-06
0.0007768922063741135
